## Analyze an Re sweep for Specific End States
### Single File: In-Line
### Monitor group velocity, distance, and angle between swimmers

In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Fri Feb 14 14:16:04 2020

@author: thomas
"""

#MODULES
import os,sys
import re
import numpy as np
import pandas as pd
from mpl_toolkits import mplot3d
%matplotlib notebook
import matplotlib as mpl
#mpl.use('Agg')
import matplotlib.pyplot as plt
from matplotlib.ticker import (MultipleLocator,AutoMinorLocator)
from scipy.signal import savgol_filter
import pathlib
from matplotlib import animation
from IPython.display import display, Image, HTML

mpl.rcParams['axes.linewidth'] = 1.5 #set the value globally

In [2]:
#CONSTANTS
cwd_PYTHON = os.getcwd()
PERIOD = 0.1
DT = 5.0e-3
RADIUSLARGE = 0.002
RADIUSSMALL = 0.001
FREQUENCY   = 10.0  

#Lists
#RLength
ReList=['0.5','0.6','0.7','0.8','0.9','1.0','2.0','3.0','4.0','5.0','5.5','6.0','6.5','7.0','7.5','10.0','12.5','15.0']
allData = []

#### Obtain Pos Data for all simulations
#### Limit time to first five oscillations

In [3]:
def StoreData(cwd_POSDATA):
    #global axAll
    #Reset position data every Sim
    pdData = []
    #Load position data
    pdData = pd.read_csv(cwd_POSDATA+'/pd.txt',delimiter=' ')
    #Save only every 20 rows (Every period)
    pdData = pdData.iloc[::20]
    #Reindex so index corresponds to period number
    pdData = pdData.reset_index(drop=True)
    #Print pdData to make sure it has been done properly
    #print(pdData.head(6))
    #Create Swimmer A and Swimmer B dataframes
    dict_A = {'xU':pdData['aXU'],'yU':pdData['aYU'],'xL':pdData['aXL'],'yL':pdData['aYL'],'time':pdData['time']}
    dataA  = pd.DataFrame(data=dict_A)
    dict_B = {'xU':pdData['bXU'],'yU':pdData['bYU'],'xL':pdData['bXL'],'yL':pdData['bYL'],'time':pdData['time']}
    dataB  = pd.DataFrame(data=dict_B)
    #print(dataA)
    #print(dataB)
    #Create a swimmer class for each
    swimA = Swimmer(dataA)
    swimB = Swimmer(dataB)
    simTime = swimA.time
    angleA = swimA.get_theta()
    angleB = swimB.get_theta()
    #print('Theta_A = ',angleA*180.0/np.pi)
    #print('Theta_B = ',angleB*180.0/np.pi)
    
    #Calculate Combined Swimmer Lab Angle
    theta_comb = CalcCombinedLabAngle(swimA,swimB)
    #Calculate Combined CM
    xCM_comb, yCM_comb = CalcSystemCM(swimA.CM,swimB.CM)
    
    #Calculate the three parameters
    #Hx, Hy, and theta_BW in swimmer A's reference frame
    Hx, Hy, Hmag, theta_BW = swimB.CalcHxHyTheta(swimA)
    #print('Lab Frame')
    labHx, labHy = swimB.get_labH()
    #print('labHx = ',labHx)
    #print('labHy = ',labHy)
    #print('\n\n')
    #print('Hx = ',Hx)
    #print('Hy = ',Hy)
    #print('Theta_BW = ',180.0*theta_BW/np.pi)
    #First, create a dataframe to contain this information
    dict_Param = {'Hx':Hx,'Hy':Hy,'Hmag':Hmag,'Theta':theta_BW,'xCM_comb':xCM_comb,'yCM_comb':yCM_comb,'time':simTime}
    parData = pd.DataFrame(data=dict_Param)
    parData['theta_deg'] = 180.0*parData['Theta']
    #print(parData)
    
    #Calculate Velocity of combined CM
    parData['vx_comb'], parData['vy_comb'], parData['v_comb'] = 0.0, 0.0, 0.0
    for idx in range(1,len(parData['time'])):
        parData.loc[idx,'vx_comb'] = (parData.loc[idx,'xCM_comb'] - parData.loc[idx-1,'xCM_comb'])/(RADIUSSMALL*PERIOD*FREQUENCY)
        parData.loc[idx,'vy_comb'] = (parData.loc[idx,'yCM_comb'] - parData.loc[idx-1,'yCM_comb'])/(RADIUSSMALL*PERIOD*FREQUENCY)
    parData['v_comb'] = np.hypot(parData['vx_comb'],parData['vy_comb'])
    
    #Using Combined Swimmer Lab Angle theta_comb, find v_par and v_perp given v_x and v_y
    v_lab = np.zeros((2,len(parData['time'])))
    v_lab[0] = np.array(parData['vx_comb'].values.tolist())
    v_lab[1] = np.array(parData['vy_comb'].values.tolist())
    #Rotate Velocities by Combined Swimmer Angle
    v_swim = np.zeros((2,len(parData['time'])))
    v_swim[0,0],v_swim[1,0] = 0.0,0.0
    for idx in range(1,len(parData['time'])):
        v_swim[:,idx] = Rotate(v_lab[:,idx],-1.0*theta_comb[idx])
    #Add V_swim to database
    parData['vpar_comb'] = v_swim[1,:]
    parData['vperp_comb'] = v_swim[0,:]
    parData['theta_comb'] = theta_comb
    print((1.0/np.pi)*180.0*parData['theta_comb'].tail())
    
    return parData


#### Find Angle Between Swimmers (Account for Quadrant)

In [4]:
def Rotate(xy, theta):
    # https://en.wikipedia.org/wiki/Rotation_matrix#In_two_dimensions
    #First Rotate based on Theta
    #Allocate Arrays
    rotationMatrix = np.zeros((2,2))
    #Calculate rotation matrix
    rotationMatrix[0,0] = np.cos(theta)
    rotationMatrix[0,1] = -1.0*np.sin(theta)
    rotationMatrix[1,0] = np.sin(theta)
    rotationMatrix[1,1] = np.cos(theta) 
    return rotationMatrix.dot(xy)

def Translate(xy, offset):
    return xy + offset

def CalcSystemCM(ACM,BCM):
    xCM_comb = 0.5*(ACM[0]+BCM[0])
    yCM_comb = 0.5*(ACM[1]+BCM[1])
    return (xCM_comb, yCM_comb)

def CalcCombinedLabAngle(A,B):
    norm_comb = np.zeros((2,len(A.xU)))
    UxCM = 0.5*(A.xU + B.xU)
    UyCM = 0.5*(A.yU + B.yU)
    LxCM = 0.5*(A.xL + B.xL)
    LyCM = 0.5*(A.yL + B.yL)
    LabX = UxCM - LxCM
    LabY = UyCM - LyCM
    Length = np.hypot(LabX,LabY)
    norm_comb[0,:] = LabX/Length
    norm_comb[1,:] = LabY/Length
    theta_comb = np.arctan2(norm_comb[1,:],norm_comb[0,:]) - np.pi/2.0 #Reference angle is when norm points straight up
    return (theta_comb)
    

class Swimmer:
    def __init__(self,data):
        self.data  = data
        self.xU, self.yU = data['xU'], data['yU']
        self.xL, self.yL = data['xL'], data['yL']
        self.time = data['time']/2.0
        self.CM = np.zeros((2,len(self.time)))
        self.theta = np.zeros(len(self.time))
        self.normY = np.zeros((2,len(self.time)))
        self.normX = np.zeros((2,len(self.time)))
        self.labH = np.zeros((2,len(self.time)))
        self.Hx = np.zeros(len(self.time))
        self.Hy = np.zeros(len(self.time))
        self.rot_norm = np.zeros((2,len(self.time)))
        self.theta_BW = np.zeros(len(self.time))
        self.CalcCM()
        #print('CM calculated')
        self.CalcLabAngle()
        #print('theta Calculated')
        #print('theta = ',self.theta)
    
    def get_theta(self):
        return self.theta

    def get_labH(self):
        return (self.labHx/RADIUSSMALL, self.labHy/RADIUSSMALL)
    
    def get_norms(self):
        return (self.normX, self.normY)
        
    def CalcCM(self):
        self.xCM = 0.8*self.xU + 0.2*self.xL
        self.yCM = 0.8*self.yU + 0.2*self.yL
        self.CM[0] = self.xCM
        self.CM[1] = self.yCM
        #print('CM = ',self.CM)
        
    def CalcLabAngle(self):
        #Find swimming axis (normal y-axis)
        self.labX   = self.xU - self.xL
        self.labY   = self.yU - self.yL
        self.length = np.hypot(self.labX,self.labY)
        self.normY[0] = self.labX/self.length
        self.normY[1] = self.labY/self.length
        #self.normY  = np.array([self.labX/self.length,self.labY,self.length])
        #2) Calculate Theta
        for idx in range(len(self.time)):
            if(self.normY[1,idx] >= 0.0):
                self.theta[idx] = np.arccos(self.normY[0,idx])
                #Theta_a[idx] = np.arccos(norm_aX[idx])
            else:
                self.theta[idx] = -1.0*np.arccos(self.normY[0,idx])+2.0*np.pi
                #Theta_a[idx] = -1.0*np.arccos(norm_aX[idx])+2.0*np.pi
        #Calculate perpendicular axis (swimmer's x-axis)
        self.normX[0] = np.cos(self.theta - 0.5*np.pi)
        self.normX[1] = np.sin(self.theta - 0.5*np.pi)
        
    def CalcHxHyTheta(self,swimmerA):
        #Here, we will calculate three parameters
        #1) Hx: In the swimmer A's reference frame
        #2) Hy: In the swimmer A's reference frame
        #3) Theta_BW: Angle between swimmer A and swimmerB (w/ respect to swimmer A y-axis)
        #But first, we must calculate in lab frame for some
        #print('yCM = ',self.yCM)
        #print('A_yCM = ',swimmerA.yCM)
        self.labHx = self.xCM - swimmerA.xCM
        self.labHy = self.yCM - swimmerA.yCM
        self.Hmag  = np.hypot(self.labHx,self.labHy)
        self.labH[0], self.labH[1] = self.labHx, self.labHy
        #1) Hx: In swimmer A's reference frame
        #2) Hy: In swimmer A's reference frame
        self.Projection(swimmerA.normX,swimmerA.normY,swimmerA.CM)
        
        #3) Rotate A and B ccw by 2pi - Theta_a
        self.rotateAngle = 2.0*np.pi - swimmerA.theta
        #RotationMatrix (def RotationMatrix)
        #Create Swimmer Position Arrays ??
        #self.normY
        
        for idx in range(len(self.length)):
            self.rot_norm[:,idx] = Rotate(self.normY[:,idx],self.rotateAngle[idx])
            swimmerA.rot_norm[:,idx] = Rotate(swimmerA.normY[:,idx],self.rotateAngle[idx])
            if(self.rot_norm[1,idx] >= 0.0):
                self.theta_BW[idx] = np.arccos(swimmerA.rot_norm[:,idx].dot(self.rot_norm[:,idx]))%(2.0*np.pi)
            else:
                self.theta_BW[idx] = (-1.0*np.arccos(swimmerA.rot_norm[:,idx].dot(self.rot_norm[:,idx]))+2.0*np.pi)%(2.0*np.pi)
        
        return (self.Hx/RADIUSSMALL, self.Hy/RADIUSSMALL, np.hypot(self.Hx,self.Hy)/RADIUSSMALL, self.theta_BW/np.pi)

    def Projection(self,normX,normY,A_CM):
        #Here we find the distance component for swimmer A's axes
        #print('normY = ',normY)
        #print('CM = ',self.CM)
        for idx in range(len(self.time)):
            '''print('normY[:,idx] = ',normY[:,idx])
            print('CM[idx] = ',self.CM[:,idx])
            dot = np.dot(normY[:,idx],self.CM[:,idx])
            print('dot = ',dot)
            ycoord = dot*normY[:,idx]
            print('ycoord = ',ycoord)'''
            self.Hx[idx] = np.dot(normX[:,idx],(self.CM[:,idx]-A_CM[:,idx]))
            self.Hy[idx] = np.dot(normY[:,idx],(self.CM[:,idx]-A_CM[:,idx]))


#### Create dataframe of all pos data for all time for all sims

In [5]:
#The main goal of this script is to create an H-Theta Phase Space of all
#simulations for every period elapsed.
#Example: 20s of sim time = 200 periods. 200 H-Theta Plots
#We may find that we can combine the H-Theta data after steady state has been reached
#1) For each simulation, store position data for every period (nothing in between)
#2) Calculate separation distance between large spheres (H)
#3) Calculate relative heading (Theta)
#4) Calculate deltaH and deltaTheta (change after one Period)
#5) Plot H vs Theta (Polar) for each period

nRe = len(ReList)

count = 0
dict_allData = {'Re':[],'Hx':[],'Hy':[],'Hmag':[],'Theta':[],'time':[]}
df_allData = pd.DataFrame(data=dict_allData)
for Re in ReList:
    cwd_POSDATA = cwd_PYTHON+'/../PosData/V/Re'+Re+'/'
    strPrint = 'Re:'+Re
    print(strPrint)
    #Get All sim data and store
    simData = StoreData(cwd_POSDATA)
    #Add Simulation Parameter identifiers
    simData['Re'] = float(Re)
    dict_sim = {'Hx':simData['Hx'],'Hy':simData['Hy'],'Hmag':simData['Hmag'],'Theta':simData['theta_deg'],
                'vx_comb':simData['vx_comb'],'vy_comb':simData['vy_comb'],'v_comb':simData['v_comb'],
                'vpar_comb':simData['vpar_comb'],'vperp_comb':simData['vperp_comb'],'par_ratio':abs(simData['vpar_comb']/simData['v_comb']),
                'Re':simData['Re'],'time':simData['time']}
    df_sim = pd.DataFrame(data=dict_sim)
    df_allData = pd.concat([df_allData,df_sim],ignore_index=True)
    count += 1
    #print('Re ='+Re)
allData_sweep = df_allData.copy()
allData_sweep = allData_sweep.sort_values(by=['Re','time'])
#allData.to_csv(cwd_PYTHON+'/allData_v2.csv',index=False,sep=' ')
print(len(allData_sweep['time']))
print('count = ',count)
print('Storing Data is complete! Onto Analysis')
                

Re:0.5
140   -204.556547
141   -204.506626
142   -204.601430
143   -204.586150
144   -204.604089
Name: theta_comb, dtype: float64
Re:0.6


/Users/thomas/anaconda3/envs/tf-research/lib/python3.6/site-packages/ipykernel_launcher.py:29: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



184   -28.418385
185   -28.420210
186   -28.443716
187   -28.444322
188   -28.464894
Name: theta_comb, dtype: float64
Re:0.7
168   -28.113225
169   -28.110467
170   -28.134325
171   -28.134834
172   -28.168846
Name: theta_comb, dtype: float64
Re:0.8
162   -27.864258
163   -27.889958
164   -27.882345
165   -27.907003
166   -27.904473
Name: theta_comb, dtype: float64
Re:0.9
196   -27.875037
197   -27.881072
198   -27.890270
199   -27.874452
200   -27.885635
Name: theta_comb, dtype: float64
Re:1.0
196   -27.358939
197   -27.358795
198   -27.365710
199   -27.348884
200   -27.358274
Name: theta_comb, dtype: float64
Re:2.0
196   -24.105666
197   -24.076071
198   -24.073095
199   -24.033537
200   -24.024758
Name: theta_comb, dtype: float64
Re:3.0
196   -32.088584
197   -32.105857
198   -32.135960
199   -32.178465
200   -32.192483
Name: theta_comb, dtype: float64
Re:4.0
196   -31.519126
197   -31.552606
198   -31.583938
199   -31.592985
200   -31.620896
Name: theta_comb, dtype: float64
Re:5.0


#### Plot the following quantities:
##### (Theta_bw vs time), (Hmag vs time), (v_comb vs time)
#### After all are found, then get plateaus (long-time behavior) and plot vs Re
#### Get mean and std of each parameter being tracked
##### (Theta_bw vs Re), (Hmag vs. Re), (v_comb vs Re)

In [6]:
#Plotting Functions

def LabelSubPlots(ax, Re):

    csfont = {'fontname':'Times New Roman'}
    ax[0].set_title(r'$\theta_{bw}$ vs. time: Re =  '+Re,fontsize=15,**csfont)
    ax[0].set_xlabel(r'time (s)',fontsize=12,**csfont)
    ax[0].set_ylabel(r'$\theta_{bw}$',fontsize=12,**csfont)
    ax[1].set_title(r'$l_{bw}$ vs. time: Re =  '+Re,fontsize=15,**csfont)
    ax[1].set_xlabel(r'time (s)',fontsize=12,**csfont)
    ax[1].set_ylabel(r'$l_{bw}$',fontsize=12,**csfont)
    ax[2].set_title(r'v$_{comb}$ vs. time: Re =  '+Re,fontsize=15,**csfont)
    ax[2].set_xlabel(r'time (s)',fontsize=12,**csfont)
    ax[2].set_ylabel(r'v$_{comb}/rf$',fontsize=12,**csfont)
    ax[3].set_title(r'v$_{//,comb}$/$v_{comb}$ vs. time: Re =  '+Re,fontsize=15,**csfont)
    ax[3].set_xlabel(r'time (s)',fontsize=12,**csfont)
    ax[3].set_ylabel(r'v$_{//,comb}$/$v_{comb}$',fontsize=12,**csfont)
    ax[0].set_xlim(0.0,20.0)
    ax[0].set_ylim(0.0,360.0)
    ax[1].set_xlim(0.0,20.0)
    ax[1].set_ylim(2.5,15.0)
    ax[2].set_xlim(0.0,20.0)
    ax[2].set_ylim(0.0,0.8)
    ax[3].set_xlim(0.0,20.0)
    ax[3].set_ylim(0.5,1.1)
    #ax[0].axis([-1.0,16.0,-19.0,16.0])
    #ax[1].axis([-1.0,16.0,-19.0,16.0])
    #ax[2].axis([-1.0,16.0,-19.0,16.0])
    
    return ax

def PlotvsTime(ax,data,varName,Re):
    ax.scatter(data['time'],data[varName],c='k',s=9)
    ax.plot(data['time'],data[varName],c='k')
    return ax
    

In [7]:
%matplotlib inline

#Plot 1) Theta_bw vs time for each Re
#     2) Dist_bw vs time for each Re
#     3) vel_combined vs time for each Re (Might need to extract new info for this)
#fig, ax = plt.subplots(nrows=1,ncols=3,num=1,figsize=(19,6),dpi=250)
for Re in ReList:
    fig, ax = plt.subplots(nrows=1,ncols=4,num=1,figsize=(25,6),dpi=250)
    ReData = allData_sweep[allData_sweep['Re'] == float(Re)].copy()
    if(float(Re) == 35.0):
        ReData = ReData[ReData['time'] <= 18.0].copy()
    ReData = ReData.reset_index(drop=True)
    for idx in range(len(ReData['time'])):
        if(ReData.loc[idx,'Theta'] <= 90.0):
            ReData.loc[idx,'Theta'] *= -1.0
        else:
            ReData.loc[idx,'Theta'] = 360.0 - ReData.loc[idx,'Theta']
    #print(ReData.head())
    ax = LabelSubPlots(ax,Re)
    ax[0] = PlotvsTime(ax[0],ReData,'Theta',Re)
    ax[1] = PlotvsTime(ax[1],ReData,'Hmag',Re)
    ax[2] = PlotvsTime(ax[2],ReData,'v_comb',Re)
    ax[3] = PlotvsTime(ax[3],ReData,'par_ratio',Re)
    fig.tight_layout()
    #plt.show()
    #sys.exit(0)
    pathlib.Path(cwd_PYTHON+'/../Figures/V/vsTime/').mkdir(parents=True, exist_ok=True)
    fig.savefig(cwd_PYTHON+'/../Figures/V/vsTime/MonitoredParameters_Re'+Re+'.png')
    fig.clf()
    print('Re = %s done'%Re)
plt.close()


Re = 0.5 done
Re = 0.6 done
Re = 0.7 done
Re = 0.8 done
Re = 0.9 done
Re = 1.0 done
Re = 2.0 done
Re = 3.0 done
Re = 4.0 done
Re = 5.0 done
Re = 5.5 done
Re = 6.0 done
Re = 6.5 done
Re = 7.0 done
Re = 7.5 done
Re = 10.0 done
Re = 12.5 done
Re = 15.0 done
Re = 17.5 done
Re = 20.0 done
Re = 25.0 done
Re = 30.0 done
Re = 35.0 done
Re = 40.0 done
Re = 50.0 done
Re = 60.0 done


In [8]:
from scipy import stats

#Construct time_ES list
time_ES = [2.5,10.0,10.0,8.5,8.0,7.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,3.5,3.0,3.0,2.5,2.5,2.5,2.5,2.0,2.0,2.0,2.0,2.0]
#Given the time_ES (time the end state begins), calculate the mean and std dev of the following parameters
#theta_BW, H_BW, v_comb for each Re
#Save them in np arrays
#Plot parameters vs Re

def GetStats(data, delta):
    #Get mean and std of 
    # Theta_BW, H_BW, and Vcomb
    #Remove any outliers (H_BW jump)
    data['zscore'] = np.abs(stats.zscore(data['Hmag']))
    data = data[data['zscore'] < 3]
    data = data.reset_index(drop=True)
    for jdx in range(len(data['time'])):
        if(data.loc[jdx,'Theta'] <= 90.0):
            data.loc[jdx,'Theta'] *= -1.0
        else:
            data.loc[jdx,'Theta'] = 360.0 - data.loc[jdx,'Theta']
    #Means and stds
    theta_avg = data['Theta'].mean()
    theta_std = data['Theta'].std()
    HBW_avg   = data['Hmag'].mean()*(delta/RADIUSSMALL)
    HBW_std   = data['Hmag'].std()*(delta/RADIUSSMALL)
    Vcomb_avg = data['v_comb'].mean()
    Vcomb_std = data['v_comb'].std()
        
    return (theta_avg, theta_std, HBW_avg, HBW_std, Vcomb_avg, Vcomb_std)
    

#Arrays (0 = mean, 1 = std dev)
thetaBW_stats = np.zeros((26,2))
HBW_stats     = np.zeros((26,2))
Vcomb_stats   = np.zeros((26,2))
theta2_stats  = np.zeros((2,2))
HBW2_stats    = np.zeros((2,2))
Vcomb2_stats  = np.zeros((2,2))

for idx in range(len(ReList)):
    ReData = allData_sweep[allData_sweep['Re'] == float(ReList[idx])].copy()
    #Calculate Delta for each Re
    delta = np.sqrt(RADIUSSMALL*RADIUSLARGE*0.8*0.8/float(ReList[idx]))
    if(float(ReList[idx]) <= 35.0):
        ReData = ReData[ReData['time'] >= 10.0].copy()
        if(float(ReList[idx]) == 35.0):
            ReData = ReData[ReData['time'] <= 18.0].copy()
        ReData = ReData.reset_index(drop=True)
        #Means and stds
        thetaBW_stats[idx,0], thetaBW_stats[idx,1], HBW_stats[idx,0], HBW_stats[idx,1], Vcomb_stats[idx,0], Vcomb_stats[idx,1] = GetStats(ReData, delta)
        
    elif(float(ReList[idx]) == 40.0):
        ReData1 = ReData[ReData['time'] >= 8.0].copy()
        ReData1 = ReData1[ReData1['time'] <= 9.0].copy()
        ReData2 = ReData[ReData['time'] >= 12.5].copy()
        ReData2 = ReData2[ReData2['time'] <= 13.5].copy()
        ReData1 = ReData1.reset_index(drop=True)
        ReData2 = ReData2.reset_index(drop=True)
        #Means and stds
        thetaBW_stats[idx,0], thetaBW_stats[idx,1], HBW_stats[idx,0], HBW_stats[idx,1], Vcomb_stats[idx,0], Vcomb_stats[idx,1] = GetStats(ReData1, delta)
        theta2_stats[0,0], theta2_stats[0,1], HBW2_stats[0,0], HBW2_stats[0,1], Vcomb2_stats[0,0], Vcomb2_stats[0,1] = GetStats(ReData2, delta)
        print(theta2_stats[0,0])
        
    elif(float(ReList[idx]) == 50.0):
        ReData1 = ReData[ReData['time'] >= 12.5].copy()
        ReData1 = ReData1[ReData1['time'] <= 17.5].copy()
        ReData2 = ReData[ReData['time'] >= 4.5].copy()
        ReData2 = ReData2[ReData2['time'] <= 8.5].copy()
        ReData1 = ReData1.reset_index(drop=True)
        ReData2 = ReData2.reset_index(drop=True)
        #Means and stds
        thetaBW_stats[idx,0], thetaBW_stats[idx,1], HBW_stats[idx,0], HBW_stats[idx,1], Vcomb_stats[idx,0], Vcomb_stats[idx,1] = GetStats(ReData1, delta)
        theta2_stats[1,0], theta2_stats[1,1], HBW2_stats[1,0], HBW2_stats[1,1], Vcomb2_stats[1,0], Vcomb2_stats[1,1] = GetStats(ReData2, delta)

    else:
        print('Re = 60 has no stable end state: Collision') 
        
    if(float(ReList[idx]) < 35.0):
        Vcomb_stats[idx,0] *= -1.0
    '''    
    ReData = ReData[ReData['time'] >= time_ES[idx]].copy()
    if(float(ReList[idx]) == 35.0):
        ReData = ReData[ReData['time'] <= 18.0].copy()
    ReData = ReData.reset_index(drop=True)
    #Remove any outliers (H_BW jump)
    ReData['zscore'] = np.abs(stats.zscore(ReData['Hmag']))
    ReData = ReData[ReData['zscore'] < 3]
    ReData = ReData.reset_index(drop=True)
    for jdx in range(len(ReData['time'])):
        if(ReData.loc[jdx,'Theta'] <= 90.0):
            ReData.loc[jdx,'Theta'] *= -1.0
        else:
            ReData.loc[jdx,'Theta'] = 360.0 - ReData.loc[jdx,'Theta']
    #Calculate Delta for each Re
    delta = np.sqrt(RADIUSSMALL*RADIUSLARGE*0.8*0.8/float(ReList[idx]))
    #Means and stds
    thetaBW_stats[idx,0] = ReData['Theta'].mean()
    thetaBW_stats[idx,1] = ReData['Theta'].std()
    HBW_stats[idx,0]     = ReData['Hmag'].mean()*(delta/RADIUSSMALL)
    HBW_stats[idx,1]     = ReData['Hmag'].std()*(delta/RADIUSSMALL)
    Vcomb_stats[idx,0]   = ReData['v_comb'].mean()
    Vcomb_stats[idx,1]   = ReData['v_comb'].std()
    if(float(ReList[idx]) < 35.0):
        Vcomb_stats[idx,0] *= -1.0
    '''

#Plot parameters vs. Re (include std as error bar)
csfont = {'fontname':'Times New Roman'}
fig, ax = plt.subplots(nrows=1,ncols=3,num=2,figsize=(19,6),dpi=250)
ax[0].set_title(r'$\theta_{bw}$ vs. Re: 2$\sigma$',fontsize=15,**csfont)
ax[0].set_xlabel(r'Re',fontsize=12,**csfont)
ax[0].set_ylabel(r'$\theta_{bw}$',fontsize=12,**csfont)
ax[1].set_title(r'$H_{bw}$ vs. Re: 2$\sigma$',fontsize=15,**csfont)
ax[1].set_xlabel(r'Re',fontsize=12,**csfont)
ax[1].set_ylabel(r'$H_{bw}$ ($\delta$)',fontsize=12,**csfont)
ax[2].set_title(r'v$_{comb}$ vs. Re: 2$\sigma$',fontsize=15,**csfont)
ax[2].set_xlabel(r'Re',fontsize=12,**csfont)
ax[2].set_ylabel(r'v$_{comb}/rf$',fontsize=12,**csfont)
#Plot Parameters with error bars
floatRe = [float(a) for a in ReList]
nRe = len(floatRe) - 1
ax[0].errorbar(floatRe[:nRe], thetaBW_stats[:nRe,0], yerr=2.0*thetaBW_stats[:nRe,1],c='k')
ax[0].scatter(floatRe[:nRe], thetaBW_stats[:nRe,0],c='k',s=12)
ax[1].errorbar(floatRe[:nRe], HBW_stats[:nRe,0], yerr=2.0*HBW_stats[:nRe,1],c='k')
ax[1].scatter(floatRe[:nRe], HBW_stats[:nRe,0],c='k',s=12)
ax[2].errorbar(floatRe[:nRe], Vcomb_stats[:nRe,0], yerr=2.0*Vcomb_stats[:nRe,1],c='k')
ax[2].scatter(floatRe[:nRe], Vcomb_stats[:nRe,0],c='k',s=12)
#Add meta-stable values
ax[0].errorbar([40.0], theta2_stats[0,0], yerr=2.0*theta2_stats[0,1],c='b')
ax[0].scatter([40.0], theta2_stats[0,0],c='b',s=12)
ax[1].errorbar([40.0], HBW2_stats[0,0], yerr=2.0*HBW2_stats[0,1],c='b')
ax[1].scatter([40.0], HBW2_stats[0,0],c='b',s=12)
ax[2].errorbar([40.0], Vcomb2_stats[0,0], yerr=2.0*Vcomb2_stats[0,1],c='b')
ax[2].scatter([40.0], Vcomb2_stats[0,0],c='b',s=12)
ax[0].errorbar([50.0], theta2_stats[1,0], yerr=2.0*theta2_stats[1,1],c='b')
ax[0].scatter([50.0], theta2_stats[1,0],c='b',s=12)
ax[1].errorbar([50.0], HBW2_stats[1,0], yerr=2.0*HBW2_stats[1,1],c='b')
ax[1].scatter([50.0], HBW2_stats[1,0],c='b',s=12)
ax[2].errorbar([50.0], Vcomb2_stats[1,0], yerr=2.0*Vcomb2_stats[1,1],c='b')
ax[2].scatter([50.0], Vcomb2_stats[1,0],c='b',s=12)

ax[0].axis([0.0,55.0,0.0,270.0])
ax[1].axis([0.0,55.0,0.5,20.0])
ax[2].axis([0.0,55.0,-0.5,0.75])
fig.tight_layout()
pathlib.Path(cwd_PYTHON+'/../Figures/V/').mkdir(parents=True, exist_ok=True)
fig.savefig(cwd_PYTHON+'/../Figures/V/Params_vs_Re_delta_v2.png')
fig.clf()
plt.close()
print(thetaBW_stats[:,0])
print(HBW_stats[:,0])
print('Plotting vs. Re is complete!')

#Plot Log-Log H_BW vs. Re

deltaValues = [np.sqrt(RADIUSSMALL*RADIUSLARGE*0.8*0.8/float(a)) for a in ReList]

#Find Relationship between H_bw and \delta
#Use HBW_stats and deltaValues
deltaArray = np.array(deltaValues)
nRe = len(deltaArray)
linearDelta = deltaArray[1:nRe-1].copy()
print(linearDelta)
linearHBW = HBW_stats[1:nRe-1,0]

#Find Linear Regression
slope, intercept, r_value, p_value, std_err = stats.linregress(np.log10(linearDelta),np.log10(linearHBW))
print("="*40)
print(r'Power Law: $\delta$ exponent')
print('N = ',len(linearDelta))
print('exponent = %.3f'%slope)
print('intercept = %.3f'%intercept)
print('r_value = %.3f'%r_value)

fig, ax = plt.subplots(nrows=1,ncols=1,num=2,figsize=(6,6),dpi=250)
ax.set_title(r'$H_{bw}$ vs. Re: log-log: $H_{bw} \propto$ $\delta^{%.2f}$'%slope,fontsize=15,**csfont)
ax.set_xlabel(r'Re$\propto$ $\delta^{-2}$',fontsize=12,**csfont)
#ax.set_xlabel(r'$\delta$ (m)',fontsize=12,**csfont)
ax.set_ylabel(r'$H_{bw}$ ($\delta$)',fontsize=12,**csfont)
#ax.loglog(deltaValues,HBW_stats[:,0],c='k')
#ax.scatter(deltaValues,HBW_stats[:,0],c='k')
ax.loglog([float(a) for a in ReList],HBW_stats[:,0],c='k')
ax.scatter([float(a) for a in ReList],HBW_stats[:,0],c='k')
#Construct Fit
#yFit = (slope*deltaArray + intercept)#*(RADIUSSMALL*RADIUSLARGE*0.8*0.8/deltaArray**3)
yFit = 10.0**(intercept)*deltaArray**slope
ax.plot([float(a) for a in ReList],yFit,c='r',ls='--')
ax.axis([0.5,60.0,0.5,20.0])
fig.tight_layout()
pathlib.Path(cwd_PYTHON+'/../Figures/V/').mkdir(parents=True, exist_ok=True)
fig.savefig(cwd_PYTHON+'/../Figures/V/H_vs_Re_LL_delta_v2.png')
fig.clf()
plt.close()
print('Plotting log-log is complete')

237.82625414274423
Re = 60 has no stable end state: Collision
[180.09892524  65.95075957  64.56026039  63.54002717  62.83949667
  61.8275493   53.88553476  46.50801982  40.04586625  35.40131712
  33.73197894  32.5395472   31.57672414  30.828941    30.18803429
  28.01488088  26.30636703  25.48477481  24.40688458  23.185694
  20.30017813  15.2172058   13.85120979  10.71209433  11.05847754
   0.        ]
[18.19641804 18.30454046 16.35789623 14.8576796  13.65877121 12.66597154
  7.63120221  5.57765464  4.40161397  3.64339538  3.36433325  3.13267186
  2.93883663  2.77364268  2.63151024  2.13350552  1.82224634  1.60491231
  1.44520875  1.32518473  1.15367439  1.0286231   0.89627307  0.83942474
  0.75106079  0.        ]
Plotting vs. Re is complete!
[0.00146059 0.00135225 0.00126491 0.00119257 0.00113137 0.0008
 0.0006532  0.00056569 0.00050596 0.00048242 0.00046188 0.00044376
 0.00042762 0.00041312 0.00035777 0.00032    0.00029212 0.00027045
 0.00025298 0.00022627 0.00020656 0.00019124 0.0001

In [38]:
#Compare v_comb to single swimmer velocity
cwd_Single = cwd_PYTHON+'/../../SingleSwimmer/Python/'
singleData = pd.read_csv(cwd_Single+'SingleSwimmer_VelData.csv',delimiter=',')
singleData = singleData[singleData['Re'] <= 50.0].copy()
singleData = singleData.reset_index(drop=True)
#Plot Both v_comb and v_single on same axes
fig, ax = plt.subplots(nrows=1,ncols=1,num=3,figsize=(6,6),dpi=250)
ax.set_title(r'$v_{avg}$ vs. Re: Pair vs. Single',fontsize=15,**csfont)
ax.set_xlabel(r'Re',fontsize=12,**csfont)
#ax.set_xlabel(r'$\delta$ (m)',fontsize=12,**csfont)
ax.set_ylabel(r'$\langle v \rangle$ (v/rf)',fontsize=12,**csfont)
nRe = len(floatRe)-1
ax.errorbar(floatRe[:nRe], Vcomb_stats[:nRe,0], yerr=2.0*Vcomb_stats[:nRe,1],c='k')
ax.scatter(floatRe[:nRe], Vcomb_stats[:nRe,0],c='k',s=12)
#Add meta-stable values
ax.errorbar([40.0], Vcomb2_stats[0,0], yerr=2.0*Vcomb2_stats[0,1],c='b')
ax.scatter([40.0], Vcomb2_stats[0,0],c='b',s=12)
ax.errorbar([50.0], Vcomb2_stats[1,0], yerr=2.0*Vcomb2_stats[1,1],c='b')
ax.scatter([50.0], Vcomb2_stats[1,0],c='b',s=12)
#Single Bot
ax.errorbar(singleData['Re'],singleData['v_avg'],yerr=2.0*singleData['v_std'],c='r')
ax.scatter(singleData['Re'],singleData['v_avg'],c='r',s=12)
ax.axis([0.0,65.0,-0.5,0.75])
fig.tight_layout()
pathlib.Path(cwd_PYTHON+'/../Figures/V/').mkdir(parents=True, exist_ok=True)
fig.savefig(cwd_PYTHON+'/../Figures/V/CompareVel_SinglePair.png')
fig.clf()
plt.close()
print('Plotting Pair vs. Single Velocities is complete')



Plotting Pair vs. Single Velocities is complete


In [36]:
%matplotlib inline
#Plot Avg Vel and Theta_BW on the same plot
#Using two different axes
fig = plt.figure(num=1,figsize=(6,4),dpi=200)
ax = fig.add_subplot(111)
csfont = {'fontname':'Times New Roman'}
#ax.set_title(r'Center of Mass',fontsize=20)
ax.set_xlabel(r'Re',fontsize=14,**csfont)
ax.set_ylabel(r'$\langle v \rangle$ (v/rf)',fontsize=14,**csfont)
#Horizontal Lines
ax.plot([0.0,65.0],[0.0,0.0] ,color='k',lw=2)
ax.errorbar(floatRe[:nRe],Vcomb_stats[:nRe,0],yerr=2.0*Vcomb_stats[:nRe,1],c='k',zorder=5)
ax.scatter(floatRe[:nRe],Vcomb_stats[:nRe,0],color='k',s=12,zorder=5)
ax2 = ax.twinx()
ax2.set_ylabel(r'$\theta_{BW}$',fontsize=14,**csfont,color='tab:blue')
nRe = len(floatRe)-1
ax2.errorbar(floatRe[:nRe],thetaBW_stats[:nRe,0],yerr=2.0*thetaBW_stats[:nRe,1],c='tab:blue',zorder=1)
ax2.scatter(floatRe[:nRe],thetaBW_stats[:nRe,0],c='tab:blue',s=12,zorder=1)
ax2.tick_params(which='major',labelcolor='tab:blue')
ax2.tick_params(which='minor',labelcolor='tab:blue')
ax.set_xlim(0.0,65.0)
ax.set_ylim(-0.5,0.75)
ax2.set_ylim(0.0,185.0)
fig.tight_layout()
pathlib.Path(cwd_PYTHON+'/../Figures/V/').mkdir(parents=True, exist_ok=True)
fig.savefig(cwd_PYTHON+'/../Figures/V/AvgVel_ThetaBW_Plot.png')
ax.set_xlim(0.0,20.0)
ax.set_ylim(-0.5,0.0)
ax2.set_ylim(0.0,90.0)
fig.savefig(cwd_PYTHON+'/../Figures/V/AvgVel_ThetaBW_Zoom.png')
fig.clf()
plt.close()
print('Plotting Avg Vel and Theta_BW together is complete')

Plotting Avg Vel and Theta_BW together is complete
